 # Code was run on https://www.kaggle.com/

# Hyperparameter tuning for DQN using Optuna

In [1]:
!pip install vector-velocity-gym

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 85.1 MB/s eta 0:00:00


In [2]:
import os
import torch
import optuna
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from gym_vectorvelocity import VectorVelocityEnv
from stable_baselines3.common.env_checker import check_env
from gymnasium import make

2024-07-25 15:15:30.935836: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 15:15:30.935989: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 15:15:31.062558: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Setup

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [4]:
n_cores = os.cpu_count()
print(f"Using {n_cores} cores for training.")

Using 4 cores for training.


### Initialize environment

In [5]:
# env modifications if needed
GAMEOVER_PENALTY = 75
MISSED_COIN_PENALTY = 3

DODGED_OBSTACLE_REWARD = 1
COLLECTED_COIN_REWARD = 12

def create_env():
    env = VectorVelocityEnv()
    env.coin_missed_penalty = MISSED_COIN_PENALTY
    env.game_over_penalty = GAMEOVER_PENALTY
    env.dodged_obstacle_reward = DODGED_OBSTACLE_REWARD
    env.coin_reward = COLLECTED_COIN_REWARD
    return env

env = create_env()
check_env(env)

In [6]:
env = make('VectorVelocity-v0')
env = DummyVecEnv([lambda: env])

### Define trial

In [7]:
def objective(trial):

    # Suggest hyperparameters
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    gamma = trial.suggest_float('gamma', 0.9, 0.9999)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    buffer_size = trial.suggest_int('buffer_size', 10000, 100000)
    exploration_fraction = trial.suggest_float('exploration_fraction', 0.1, 0.5)
    exploration_final_eps = trial.suggest_float('exploration_final_eps', 0.01, 0.1)
    
    # Create the model
    model = DQN(
        "MultiInputPolicy",
        env,
        learning_rate=learning_rate,
        gamma=gamma,
        batch_size=batch_size,
        buffer_size=buffer_size,
        exploration_fraction=exploration_fraction,
        exploration_final_eps=exploration_final_eps,
        verbose=1
    )
    
    # Train the model
    model.learn(total_timesteps=100000)
    
    # Evaluate the model
    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=20, deterministic=True)
    
    return mean_reward

### Start optimization

In [8]:
#Create the Optuna study with a storage URL
storage_url = "sqlite:///hyperparameter_tuning_data.db"
study = optuna.create_study(storage=storage_url, direction='maximize', study_name='dqn_hyperparameter_tuning')

# Run the optimization with multiprocessing
study.optimize(objective, n_trials=50, n_jobs=n_cores)

# Print the best hyperparameters
print("Best hyperparameters: ", study.best_params)

[I 2024-07-25 15:15:43,856] A new study created in RDB with name: dqn_hyperparameter_tuning


Using cuda device
Using cuda device
Using cuda device
Using cuda device


/opt/conda/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:181: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.782    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 61       |
|    time_elapsed     | 80       |
|    total_timesteps  | 4961     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.694    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 61       |
|    time_elapsed     | 119      |
|    total_timesteps  | 7384     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.672    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 61       |
|    time_elapsed     | 162      |
|    total_timesteps  | 10034    |
----------------------------------
----------------------------------
| rollout/          

/opt/conda/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
[I 2024-07-25 16:15:22,173] Trial 1 finished with value: 226.9585200035903 and parameters: {'learning_rate': 0.00036489688346954364, 'gamma': 0.9054250802048465, 'batch_size': 256, 'buffer_size': 86485, 'exploration_fraction': 0.29763301813026377, 'exploration_final_eps': 0.026544085130239398}. Best is trial 1 with value: 226.9585200035903.


...

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.0321   |
| time/               |          |
|    episodes         | 56       |
|    fps              | 55       |
|    time_elapsed     | 1312     |
|    total_timesteps  | 73365    |
| train/              |          |
|    learning_rate    | 4.85e-05 |
|    loss             | 0.00018  |
|    n_updates        | 5841     |
----------------------------------


[I 2024-07-26 01:04:09,548] Trial 48 finished with value: 17.62813511302362 and parameters: {'learning_rate': 0.0002141343455180411, 'gamma': 0.9895873900542063, 'batch_size': 256, 'buffer_size': 37735, 'exploration_fraction': 0.34182136669889296, 'exploration_final_eps': 0.03524029754122275}. Best is trial 0 with value: 242.18506515581117.


----------------------------------
| rollout/            |          |
|    exploration_rate | 0.0321   |
| time/               |          |
|    episodes         | 60       |
|    fps              | 55       |
|    time_elapsed     | 1365     |
|    total_timesteps  | 76152    |
| train/              |          |
|    learning_rate    | 4.85e-05 |
|    loss             | 0.000127 |
|    n_updates        | 6537     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.0321   |
| time/               |          |
|    episodes         | 64       |
|    fps              | 55       |
|    time_elapsed     | 1400     |
|    total_timesteps  | 78229    |
| train/              |          |
|    learning_rate    | 4.85e-05 |
|    loss             | 0.292    |
|    n_updates        | 7057     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rat

[I 2024-07-26 01:13:41,942] Trial 49 finished with value: -51.375477691483866 and parameters: {'learning_rate': 4.8537620328904014e-05, 'gamma': 0.990323878603262, 'batch_size': 256, 'buffer_size': 35769, 'exploration_fraction': 0.3961688424759834, 'exploration_final_eps': 0.032097756518451315}. Best is trial 0 with value: 242.18506515581117.


Best hyperparameters:  {'learning_rate': 2.280273827386646e-05, 'gamma': 0.9592251881094116, 'batch_size': 256, 'buffer_size': 14782, 'exploration_fraction': 0.22146868394466548, 'exploration_final_eps': 0.02694855175262556}


### Run dashboard to inspect results

In [2]:
from optuna_dashboard import run_server
storage_url = "sqlite:///./hyperparameter_tuning_data.db"
# Start the Optuna dashboard
dashboard_proc = run_server(storage_url, host="127.0.0.1", port=8080)

Bottle v0.12.25 server starting up (using WSGIRefServer())...
Listening on http://127.0.0.1:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [26/Jul/2024 07:53:07] "GET /dashboard/?studies_order_by=desc HTTP/1.1" 200 4145
127.0.0.1 - - [26/Jul/2024 07:53:07] "GET /static/bundle.js HTTP/1.1" 200 2986981
127.0.0.1 - - [26/Jul/2024 07:53:08] "GET /api/studies HTTP/1.1" 200 153
d:\Uni\Reinforcement_Learning_Vector_Velocity\reinforcement_learning\lib\site-packages\optuna_dashboard\_importance.py:75: ExperimentalWarning: PedAnovaImportanceEvaluator is experimental (supported from v3.6.0). The interface can change in the future.
  return get_param_importances(study, target=target, evaluator=PedAnovaImportanceEvaluator())
d:\Uni\Reinforcement_Learning_Vector_Velocity\reinforcement_learning\lib\site-packages\optuna\importance\_ped_anova\evaluator.py:150: UserWarning: PedAnovaImportanceEvaluator computes the importances of params to achieve low `target` values. If this is not what you want, please modif